In [1]:
using SymPy
using LinearAlgebra

## 1. feladat

### Setup

In [2]:
# DH matrix
function DH(α, a, d, θ)
    [cos(θ) -sin(θ) 0 a;
    cos(α)*sin(θ) cos(α)*cos(θ) -sin(α) -d*sin(α);
    sin(α)*sin(θ) sin(α)*cos(θ) cos(α) d*cos(α);
    0 0 0 1]
end

DH (generic function with 1 method)

In [65]:
# Mechanism parameters
l10_ = Sym(8//10); #m
l11_ = Sym(6//10); #m
l2_ = Sym(3//10); #m
rhoA_ = Sym(60); #kg/m
m_ = Sym(5); #kg
IE_ = Sym(1); #MNm^2

g_ = Sym(981//100); # m/s^2

In [4]:
# just for shorter Sym(0)-s
_0 = Sym(0);

In [5]:
# Symbolic variables
@vars q1 q2 l10 l11 l2

(q1, q2, l10, l11, l2)

In [66]:
@vars rhoA m IE m10 m11 m2 m20 g

(rhoA, m, IE, m10, m11, m2, m20, g)

In [7]:
# For derivatives
@vars t
q1t = SymFunction("q1");
q2t = SymFunction("q2");

In [8]:
function ssub(A, x1, x2)
    x1f = x1.from
    x1t = x1.to
    x2f = x2.from
    x2t = x2.to
    return A.subs(x1f, x1t).subs(x2f, x2t)
end

ssub (generic function with 1 method)

In [9]:
x1t = (from = q1, to = q1t(t))
x2t = (from = q2, to = q2t(t))
t1x = (from = q1t(t), to = q1)
t2x = (from = q2t(t), to = q2)
dt1 = (from = q1t'(t), to = Sym(1))
dt2 = (from = q2t'(t), to = Sym(1))

(from = Derivative(q2(t), t), to = 1)

In [10]:
function d(A, x)
    A_ = deepcopy(A)
    for i in eachindex(A)
        A_[i] = diff(A[i], x)
    end
    A_
end

d (generic function with 1 method)

In [11]:
function simpl(A)
    A_ = deepcopy(A)
    for i in eachindex(A)
        A_[i] = simplify(A[i])
    end
    A_
end

simpl (generic function with 1 method)

In [12]:
J(V, x1, x2) = hcat(Sym[diff(V[i], x1) for i in 1:3], Sym[diff(V[i], x2) for i in 1:3])

J (generic function with 1 method)

In [13]:
function steiner(m, x,y,z)
    m*[y^2+z^2 -x*y -x*z;
    -y*x x^2+z^2 -y*z;
    -z*x -z*y x^2+y^2]
end

steiner (generic function with 1 method)

In [14]:
function steiner(m,v)
    steiner(m, v[1], v[2], v[3])
end

steiner (generic function with 2 methods)

### DH paraméterek

In [15]:
# first joint
α1 = _0; a1 = _0; d1 = l10; θ1 = PI/2+q1;
# second joint
α2 = -PI/2; a2 = l11; d2 = _0; θ2 = q2;
# E TCP
α3 = PI/2; a3 = _0; d3 = l2; θ3 = -PI/2;

In [16]:
_0T1 = DH(α1, a1, d1, θ1)

4×4 Array{Sym,2}:
 -sin(q1)  -cos(q1)  0    0
  cos(q1)  -sin(q1)  0    0
        0         0  1  l10
        0         0  0    1

In [17]:
_1T2 = DH(α2, a2, d2, θ2)

4×4 Array{Sym,2}:
  cos(q2)  -sin(q2)  0  l11
        0         0  1    0
 -sin(q2)  -cos(q2)  0    0
        0         0  0    1

In [18]:
_2TTCP = DH(α3, a3, d3, θ3)

4×4 Array{Sym,2}:
  0  1   0    0
  0  0  -1  -l2
 -1  0   0    0
  0  0   0    1

### Direkt kinematika

In [19]:
_2rTCP = [0, -l2, 0, 1]

4-element Array{Sym,1}:
   0
 -l2
   0
   1

In [20]:
_0rTCP = _0T1 * _1T2 * _2rTCP

4-element Array{Sym,1}:
 -l11*sin(q1) - l2*sin(q1)*sin(q2)
  l11*cos(q1) + l2*sin(q2)*cos(q1)
                  l10 + l2*cos(q2)
                                 1

In [21]:
_0rTCP_check = _0T1 * _1T2 * _2TTCP * [0,0,0,1]

4-element Array{Sym,1}:
 -l11*sin(q1) - l2*sin(q1)*sin(q2)
  l11*cos(q1) + l2*sin(q2)*cos(q1)
                  l10 + l2*cos(q2)
                                 1

In [22]:
_0rTCP_check == _0rTCP

true

In [23]:
# Hasznos mátrixok
_0T2 = _0T1 * _1T2;

### Inverz kinematika

`_0rTCP` - ből

### J_E Jacobi-mátrix

In [24]:
J_E = J(_0rTCP, q1, q2)

3×2 Array{Sym,2}:
 -l11*cos(q1) - l2*sin(q2)*cos(q1)  -l2*sin(q1)*cos(q2)
 -l11*sin(q1) - l2*sin(q1)*sin(q2)   l2*cos(q1)*cos(q2)
                                 0          -l2*sin(q2)

### Forgatási tengely és szög

In [25]:
_0TTCP = _0T1 * _1T2 * _2TTCP

4×4 Array{Sym,2}:
 cos(q1)  -sin(q1)*cos(q2)  …  -l11*sin(q1) - l2*sin(q1)*sin(q2)
 sin(q1)   cos(q1)*cos(q2)      l11*cos(q1) + l2*sin(q2)*cos(q1)
       0          -sin(q2)                      l10 + l2*cos(q2)
       0                 0                                     1

In [26]:
_0RTCP_q20 = _0TTCP.subs(q2, Sym(0))[1:3, 1:3]

3×3 Array{Sym,2}:
 cos(q1)  -sin(q1)  0
 sin(q1)   cos(q1)  0
       0         0  1

## 2. feladat

## Tömegek

In [27]:
m10_ = rhoA_*l10_;
m11_ = rhoA_*l11_;
m1_ = rhoA_*(l10_+l11_);
m20_ = rhoA_*l2_;
m2_ = m20_+m_;

#### Sebesség Jacobi

In [28]:
# rúd10
_1r01 = [_0, _0, -l10/2, Sym(1)];
# rúd11
_1r11 = [l11/2, _0, _0, Sym(1)];
# rúd2
_2r2 = [_0, -l2/2, _0, Sym(1)];
# m tömeg
_2rm = [_0, -l2, _0, Sym(1)];

In [29]:
# transzformálás 0-s kr-be és összegzés a tagokra
_1rrud1 = (m10_*_1r01+m11_*_1r11)/m1_;
_0rrud1 = _0T1*_1rrud1;
_2rrud2 = (m20_*_2r2+m_*_2rm)/m2_;
_0rrud2 = _0T2*_2rrud2;

In [30]:
_0Jrs1 = J(_0rrud1, q1, q2);
N0Jrs1 = _0Jrs1.subs(l11, l11_);
N0Jrs1.evalf(4)

3×2 Array{Sym,2}:
 -0.1286*cos(q1)  0
 -0.1286*sin(q1)  0
               0  0

In [31]:
_0Jrs2 = J(_0rrud2, q1, q2);
N0Jrs2 = _0Jrs2.subs(l11, l11_).subs(l2, l2_);
N0Jrs2.evalf(5)

3×2 Array{Sym,2}:
 -0.18261*sin(q2)*cos(q1) - 0.6*cos(q1)  -0.18261*sin(q1)*cos(q2)
 -0.18261*sin(q1)*sin(q2) - 0.6*sin(q1)   0.18261*cos(q1)*cos(q2)
                                      0          -0.18261*sin(q2)

#### Szögsebesség Jacobi

##### ω1

In [32]:
_0R1 = _0T1[1:3,1:3];
_0R1t = ssub(_0R1, x1t, x2t)

3×3 Array{Sym,2}:
 -sin(q1(t))  -cos(q1(t))  0
  cos(q1(t))  -sin(q1(t))  0
           0            0  1

In [33]:
_0R1d = d(_0R1t, t)

3×3 Array{Sym,2}:
 -cos(q1(t))*Derivative(q1(t), t)   sin(q1(t))*Derivative(q1(t), t)  0
 -sin(q1(t))*Derivative(q1(t), t)  -cos(q1(t))*Derivative(q1(t), t)  0
                                0                                 0  0

In [34]:
_0ωx1 = _0R1d * permutedims(_0R1t);
_ω1 = [_0ωx1[3,2], _0ωx1[1,3], _0ωx1[2,1]];

In [35]:
_0Jω1 = simpl(J(_ω1, q1t'(t), q2t'(t)));
_0Jω1nt = ssub(_0Jω1, t1x, t2x)

3×2 Array{Sym,2}:
 0  0
 0  0
 1  0

##### ω2

In [36]:
_0R2 = _0T2[1:3,1:3];
_0R2t = ssub(_0R2,x1t, x2t)

3×3 Array{Sym,2}:
 -sin(q1(t))*cos(q2(t))   sin(q1(t))*sin(q2(t))  -cos(q1(t))
  cos(q1(t))*cos(q2(t))  -sin(q2(t))*cos(q1(t))  -sin(q1(t))
            -sin(q2(t))             -cos(q2(t))            0

In [37]:
_0R2d = d(_0R2t, t);
simpl(_0R2d)

3×3 Array{Sym,2}:
  sin(q1(t))*sin(q2(t))*Derivative(q2(t), t) - cos(q1(t))*cos(q2(t))*Derivative(q1(t), t)  …   sin(q1(t))*Derivative(q1(t), t)
 -sin(q1(t))*cos(q2(t))*Derivative(q1(t), t) - sin(q2(t))*cos(q1(t))*Derivative(q2(t), t)     -cos(q1(t))*Derivative(q1(t), t)
                                                         -cos(q2(t))*Derivative(q2(t), t)                                    0

In [38]:
_0ωx2 = _0R2d * permutedims(_0R2t);
_ω2 = [_0ωx2[3,2], _0ωx2[1,3], _0ωx2[2,1]];
simpl(_ω2)

3-element Array{Sym,1}:
 -cos(q1(t))*Derivative(q2(t), t)
 -sin(q1(t))*Derivative(q2(t), t)
             Derivative(q1(t), t)

In [39]:
_0Jω2 = simpl(J(_ω2, q1t'(t), q2t'(t)));
_0Jω2nt = ssub(_0Jω2, t1x, t2x)

3×2 Array{Sym,2}:
 0  -cos(q1)
 0  -sin(q1)
 1         0

### Tehetetlenségi nyomatékok

#### 1-es test

In [40]:
_1th10 = m10*[Sym(1//12)*l10^2 _0 _0;
            _0 Sym(1//12)*l10^2 _0;
            _0 _0 _0]

3×3 Array{Sym,2}:
 l10^2*m10/12             0  0
            0  l10^2*m10/12  0
            0             0  0

In [41]:
_1th10s = steiner(m10, _1r01-_1rrud1)

3×3 Array{Sym,2}:
    9*l10^2*m10/196                                0  -9*l10*l11*m10/196
                  0  m10*(9*l10^2/196 + 9*l11^2/196)                   0
 -9*l10*l11*m10/196                                0     9*l11^2*m10/196

In [42]:
_1th11 = m11*[_0 _0 _0;
            _0 Sym(1//12)*l11^2 _0;
            _0 _0 Sym(1//12)*l11^2]

3×3 Array{Sym,2}:
 0             0             0
 0  l11^2*m11/12             0
 0             0  l11^2*m11/12

In [43]:
_1th11s = steiner(m11, _1r11-_1rrud1)

3×3 Array{Sym,2}:
    4*l10^2*m11/49                              0  -4*l10*l11*m11/49
                 0  m11*(4*l10^2/49 + 4*l11^2/49)                  0
 -4*l10*l11*m11/49                              0     4*l11^2*m11/49

In [44]:
_1θ1s = _1th10+_1th10s+_1th11+_1th11s
simpl(_1θ1s)

3×3 Array{Sym,2}:
   l10^2*(19*m10 + 12*m11)/147  …  -l10*l11*(9*m10 + 16*m11)/196
                             0                                 0
 -l10*l11*(9*m10 + 16*m11)/196       l11^2*(27*m10 + 97*m11)/588

In [45]:
N_1θ1s = _1θ1s.subs(m10,m10_).subs(m11,m11_).subs(l10,l10_).subs(l11,l11_);
N_1θ1s.evalf(5)

3×3 Array{Sym,2}:
  5.8514       0  -2.4686
       0  8.7829        0
 -2.4686       0   2.9314

#### 2-es test

In [46]:
_2th20 = m20*[Sym(1//12)*l2^2 _0 _0;
            _0 _0 _0;
            _0 _0 Sym(1//12)*l2^2]

3×3 Array{Sym,2}:
 l2^2*m20/12  0            0
           0  0            0
           0  0  l2^2*m20/12

In [47]:
_2th20s = steiner(m20, _2r2-_2rrud2)

3×3 Array{Sym,2}:
 25*l2^2*m20/2116  0                 0
                0  0                 0
                0  0  25*l2^2*m20/2116

In [48]:
_2thms = steiner(m, _2rm-_2rrud2)

3×3 Array{Sym,2}:
 81*l2^2*m/529  0              0
             0  0              0
             0  0  81*l2^2*m/529

In [49]:
_2θ2s = _2th20+_2th20s+_2thms;
simpl(_2θ2s)

3×3 Array{Sym,2}:
 l2^2*(243*m + 151*m20)/1587  0                            0
                           0  0                            0
                           0  0  l2^2*(243*m + 151*m20)/1587

In [50]:
N_2θ2s = _2θ2s.subs(m20,m20_).subs(m,m_).subs(l2,l2_)
N_2θ2s.evalf(5)

3×3 Array{Sym,2}:
 0.22304  0        0
       0  0        0
       0  0  0.22304

### Tömegmátrixok

In [51]:
Hv1 = m1_*permutedims(N0Jrs1)*N0Jrs1;
simpl(Hv1)

2×2 Array{Sym,2}:
 243/175  0
       0  0

In [62]:
Hv2 = m2_*permutedims(N0Jrs2)*N0Jrs2;
simpl(Hv2)

2×2 Array{Sym,2}:
 9*(7*sin(q2) + 23)^2/575        0
                        0  441/575

In [53]:
Hω1 = permutedims(_0Jω1nt) * _0R1 * _1θ1s * permutedims(_0R1) * _0Jω1nt

2×2 Array{Sym,2}:
 9*l11^2*m10/196 + 97*l11^2*m11/588  0
                                  0  0

In [54]:
Hω2 = permutedims(_0Jω2nt) * _0R2 * _2θ2s * permutedims(_0R2) * _0Jω2nt;
simpl(Hω2)

2×2 Array{Sym,2}:
 l2^2*(243*m + 151*m20)*sin(q2)^2/1587                            0
                                     0  l2^2*(243*m + 151*m20)/1587

In [55]:
H = Hv1+Hv2+Hω1+Hω2;
simpl(H)

2×2 Array{Sym,2}:
 9*l11^2*m10/196 + 97*l11^2*m11/588 + 81*l2^2*m*sin(q2)^2/529 + 151*l2^2*m20*sin(q2)^2/1587 + 441*sin(q2)^2/575 + 126*sin(q2)/25 + 1692/175  …                                            0
                                                                                                                                          0     81*l2^2*m/529 + 151*l2^2*m20/1587 + 441/575

In [56]:
simpl(H.subs(m10,m10_).subs(m20,m20_).subs(m11,m11_).subs(m,m_).subs(l11,l11_).subs(l2,l2_)).evalf(5)

2×2 Array{Sym,2}:
 0.99*sin(q2)^2 + 5.04*sin(q2) + 12.6        0
                                    0  0.99000

In [57]:
H = Hv1+Hv2+Hω1+Hω2;
simpl(H)

2×2 Array{Sym,2}:
 9*l11^2*m10/196 + 97*l11^2*m11/588 + 81*l2^2*m*sin(q2)^2/529 + 151*l2^2*m20*sin(q2)^2/1587 + 441*sin(q2)^2/575 + 126*sin(q2)/25 + 1692/175  …                                            0
                                                                                                                                          0     81*l2^2*m/529 + 151*l2^2*m20/1587 + 441/575

In [58]:
simpl(H.subs(m10,m10_).subs(m20,m20_).subs(m11,m11_).subs(m,m_).subs(l11,l11_).subs(l2,l2_).evalf(5))

2×2 Array{Sym,2}:
 0.99*sin(q2)^2 + 5.04*sin(q2) + 12.6        0
                                    0  0.99000

### Kinetikus energia

In [125]:
T_ = Sym(1//2)*[q1t'(t) q2t'(t)]*H*[q1t'(t),q2t'(t)];
T = T_[1].subs(m10,m10_).subs(m20,m20_).subs(m11,m11_).subs(m,m_).subs(l11,l11_).subs(l2,l2_);

In [126]:
simplify(T.evalf(5))

                                                 2                    2
/         2                         \ /d        \          /d        \ 
\0.495*sin (q2) + 2.52*sin(q2) + 6.3/*|--(q1(t))|  + 0.495*|--(q2(t))| 
                                      \dt       /          \dt       / 

### Potenciális energia

In [136]:
U_ = m1_*[0 0 g]*_0rrud1[1:3] + m2_*[0 0 g]*_0rrud2[1:3];
U = U_[1].subs(l10,  l10_).subs(g, g_).subs(l2, l2_);
simplify(U).evalf(5)

41.202*cos(q2) + 651.38

### Másodfajú Lagrange-egyenlet

##### k=1

In [128]:
deltaT_deltaq1p = simplify(diff(T, q1t'(t)).evalf(5))

/        2                          \ d        
\0.99*sin (q2) + 5.04*sin(q2) + 12.6/*--(q1(t))
                                      dt       

In [129]:
d_dt_deltaT_deltaq1p = diff(deltaT_deltaq1p, t)

                                        2       
/        2                          \  d        
\0.99*sin (q2) + 5.04*sin(q2) + 12.6/*---(q1(t))
                                        2       
                                      dt        

##### k=2

In [141]:
deltaT_deltaq2p = simplify(diff(T, q2t'(t),t).evalf(5))

       2       
      d        
0.99*---(q2(t))
       2       
     dt        

In [132]:
deltaT_deltaq2 = simplify(diff(T, q2).evalf(5))

                                         2
                              /d        \ 
(0.99*sin(q2) + 2.52)*cos(q2)*|--(q1(t))| 
                              \dt       / 

In [138]:
deltaU_deltaq2 = diff(U, q2).evalf(5)

-41.202*sin(q2)